### Priors and explanation
Idil Sahin '26, Jan 27th 2024

#### Unconditional model

In [30]:
import jax
import jax.numpy as jnp
from memo import memo
from enum import IntEnum
from xarray import DataArray

class Gesture(IntEnum):
    SOMETHINGELSE = 0
    SALUTE = 1

class Ideology(IntEnum):
    DEMOCRACY = 0
    FASCISM = 1

class Demeanor(IntEnum):
    SUAVE = 0
    AWKWARD = 1

class Information(IntEnum): # someone DESCRIBES what the broker is doing
    BLUE = 1 ## hearing the interpretation from a democrat -> positive influence in the belief that the broker is pro-fascism, negative influence in the belief that the broker is pro-democracy
    RED = 2 ## hearing the interpretation from a conservative -> positive influence in the belief that the broker is pro-demo, negative influence in the belief that the broker is pro-fascism



@jax.jit
def gesture_pmf(gesture, ideology, demeanor, information):
    
    # hearing from a democrat -- they tend to call everyone fascist 
    # however they are more truthful in conveying information
    ### P(Gesture=SALUTE | Ideology=DEMOCRACY, Demeanor=SUAVE, Information=BLUE)
    p_salute__dem_suave_blue = 0.001
    ### P(Gesture=SALUTE | Ideology=DEMOCRACY, Demeanor=AWKWARD, Information=BLUE)
    p_salute__dem_awk_blue = 0.2
    ### P(Gesture=SALUTE | Ideology=FASCISM, Demeanor=SUAVE, Information=BLUE)
    p_salute__fasc_suave_blue = 0.80
    ### P(Gesture=SALUTE | Ideology=FASCISM, Demeanor=AWKWARD, Information=BLUE)
    p_salute__fasc_awk_blue = 0.82
    
    # hearing from a conservative -- they tend to not call fascists fascists
    # they are more relaxed in conveying information
    ### P(Gesture=SALUTE | Ideology=DEMOCRACY, Demeanor=SUAVE, Information=RED)
    p_salute__dem_suave_red = 0.0001 # <- more likely that a conservative is making polarizing statements
    ### P(Gesture=SALUTE | Ideology=DEMOCRACY, Demeanor=AWKWARD, Information=RED)
    p_salute__dem_awk_red = 0.1 # <- more likely that a conservative is making polarizing statements
    ### P(Gesture=SALUTE | Ideology=FASCISM, Demeanor=SUAVE, Information=RED)
    p_salute__fasc_suave_red = 0.98
    ### P(Gesture=SALUTE | Ideology=FASCISM, Demeanor=AWKWARD, Information=RED)
    p_salute__fasc_awk_red = 0.92 

    

    ### P(Gesture=SALUTE | Ideology, Demeanor, Information)

    p_salute = jnp.array([
    [[p_salute__dem_suave_blue, p_salute__dem_suave_red], 
     [p_salute__fasc_suave_blue, p_salute__fasc_suave_red]],

    [[p_salute__dem_awk_blue, p_salute__dem_awk_red], 
     [p_salute__fasc_awk_blue, p_salute__fasc_awk_red]],
])[demeanor, ideology, information]

    
    
    ### P(Gesture | Ideology, Demeanor)
    return jnp.array([1 - p_salute, p_salute])[gesture]

@memo
def speech_simulation[_g: Gesture, _i: Ideology, _d: Demeanor, _inf: Information](prior_fasc=0.5):
    observer: knows(_d, _i, _g, _inf)
    observer: thinks[
        broker: given(d in Demeanor, wpp=1),
        broker: chooses(i in Ideology, wpp=prior_fasc if i == {Ideology.FASCISM} else 1 - prior_fasc),
        broker: chooses(inf in Information, wpp=1), 
        broker: chooses(g in Gesture, wpp=gesture_pmf(g, i, d, inf)),
    ]

    return observer[ 
        Pr[
            broker.g == _g
            and broker.i == _i
            and broker.d == _d
            and broker.inf == _inf
        ] 
    ]

### observer 1
print(f"Observer 1, with a uniform prior belief about whether the broker is pro-fascism or pro-democracy")
res1joint = speech_simulation(prior_fasc=0.5, print_table=True, return_aux=True, return_xarray=True)
print("\n\n")

res1joint
### observer 2
print(f"Observer 2, who thinks that the broker being pro-fascism is unlikely")
res2joint = speech_simulation(prior_fasc=0.01, print_table=True, return_aux=True, return_xarray=True)

Observer 1, with a uniform prior belief about whether the broker is pro-fascism or pro-democracy
+---------------+--------------+--------------+-------------------+--------------------------------------+
| _g: Gesture   | _i: Ideology | _d: Demeanor | _inf: Information | speech_simulation[_g, _i, _d, _inf]  |
+---------------+--------------+--------------+-------------------+--------------------------------------+
| SOMETHINGELSE | DEMOCRACY    | SUAVE        | BLUE              | 0.12498749792575836                  |
| SOMETHINGELSE | DEMOCRACY    | SUAVE        | RED               | 0.12498749792575836                  |
| SOMETHINGELSE | DEMOCRACY    | AWKWARD      | BLUE              | 0.11249999701976776                  |
| SOMETHINGELSE | DEMOCRACY    | AWKWARD      | RED               | 0.11249999701976776                  |
| SOMETHINGELSE | FASCISM      | SUAVE        | BLUE              | 0.002499997615814209                 |
| SOMETHINGELSE | FASCISM      | SUAVE        |

In [39]:
@memo
def speech_observation[_g: Gesture, _i: Ideology, _d: Demeanor, inf: Information](prior_fasc=0.5):
    observer: knows(_d, _i, _g, _inf)
    observer: thinks[
        broker: given(d in Demeanor, wpp=1),
        broker: chooses(i in Ideology, wpp=prior_fasc if i == {Ideology.FASCISM} else 1 - prior_fasc),
        broker: chooses(inf in Information, wpp=1),
        broker: chooses(g in Gesture, wpp=gesture_pmf(g, i, d, inf)),
    ]


    # observe the gesture through a retellilng
    observer: observes [broker.g] is _g
    observer: observes [broker.inf] is _dem

    return observer[ 
        Pr[
            ### replace _g with {Gesture.SALUTE} to ignore 
            ### probs for when gesture something other than salute
            broker.g == Gesture.SALUTE
            and broker.i == _i
            and broker.d == _d
            and broker.inf == Information.RED  
        ] 
    ]

###
# How do these two observers' priors affect how they update their beliefs when they observe the gesture?
###

### observer 1
print(f"Observer 1, with a uniform prior belief about whether the broker is pro-fascism or pro-democracy")
res1 = speech(prior_fasc=0.5, print_table=True, return_aux=True, return_xarray=True)
res1._dem
print("\n\n")

### observer 2
print(f"Observer 2, who thinks that the broker being pro-fascism is unlikely (before observing the speech)")
res2 = speech(prior_fasc=0.01, print_table=True, return_aux=True, return_xarray=True)

memo.core.MemoError: Knowing unknown choice
  file: "3402420250.py", line 3, in @memo speech_observation
        observer: knows(_d, _i, _g, _inf)
                  ^

  hint: observer does not yet model self's choice of _inf. So, it
        doesn't make sense for observer to model observer as knowing
        that choice.

  ctxt: This error was encountered in the frame of observer.  In that
        frame, observer is currently modeling the following 7 choices:
        _g, _i, _d, inf, observer._d, observer._i, observer._g.

  info: You are using memo 0.5.1, JAX 0.4.38, Python 3.13.1 on Windows.


### Discussion Questions

1. Describe these models in terms of Bayes’ rule. What’s the prior, likelihood, and posterior in these models? What is happening mathematically when we go from the first model to the second model?


Answer: 
In the first model, we define the unconditional joint probability distribution:  

$$
P(I, D, G)
$$

where:  
- $I$ represents ideology,  
- $D$ represents demeanor,  
- $G$ represents gesture.  

Each observer has a prior belief about ideology. The first observer assumes a uniform prior $ P(I = \text{fascism}) = 0.5 $, while the second observer assumes an unlikely prior $ P(I = \text{fascism}) = 0.01 $.  

In the second model, we aim to infer the latent causes, eg. getting the **posterior probability** of ideology and demeanor given a gesture:  

$$
P(I, D \mid G) = \frac{P(I, D, G)}{P(G)}
$$

This follows directly from Bayes' rule, where:  
- The **prior** is $ P(I, D) $,  
- The **likelihood** is $ P(G \mid I, D) $ (how likely a gesture is given ideology and demeanor),  
- The **posterior** is $ P(I, D \mid G) $ (updated belief after observing the gesture).  


Mathematically, moving from model 1 to model 2 involves conditioning on the gesture. This operation normalizes the joint probability over all possible values of $ I $ and $ D $, making sure that the posterior sums to 1:  

$$
P(I, D \mid G) = \frac{P(I, D, G)}{P(G)} =\frac{P(I, D, G)}{\sum_{I', D'} P(I', D', G)}
$$



2. Are observers 1 and 2 equally rational? Explain.


As the observers correctly apply Bayes’ rule, they are equally rational in a formal sense—they are just starting from different prior beliefs. However observer 1 is more empirically rational in this specific context; considering the multiple far-right statements and actions of Elon Musk (a few examples: [anti-semitism](https://www.theatlantic.com/ideas/archive/2023/05/elon-musk-george-soros-anti-semites/674072/), [racism](https://www.bbc.com/news/articles/cred8770008o)) which are closely related to having a fascist ideology. 
  


### Computational Questions

3. Adjust the likelihood and prior probabilities to match your beliefs about different people. Explain your adjustments and the effects. Did your adjustments bring the cognition predicted by the model closer to the patterns of cognition you were targeting?


In [1]:
import jax
import jax.numpy as jnp
from memo import memo
from enum import IntEnum

class Gesture(IntEnum):
    SOMETHINGELSE = 0
    SALUTE = 1

class Ideology(IntEnum):
    DEMOCRACY = 0
    FASCISM = 1

class Demeanor(IntEnum):
    SUAVE = 0
    AWKWARD = 1

@jax.jit
def gesture_pmf(gesture, ideology, demeanor):
    ### P(Gesture=SALUTE | Ideology=DEMOCRACY, Demeanor=SUAVE)
    p_salute__dem_suave = 0.02
    ### P(Gesture=SALUTE | Ideology=DEMOCRACY, Demeanor=AWKWARD)
    p_salute__dem_awk = 0.3
    ### P(Gesture=SALUTE | Ideology=FASCISM, Demeanor=SUAVE)
    p_salute__fasc_suave = 0.97
    ### P(Gesture=SALUTE | Ideology=FASCISM, Demeanor=AWKWARD)
    p_salute__fasc_awk = 0.95 

    ### P(Gesture=SALUTE | Ideology, Demeanor)
    p_salute = jnp.array([
        [p_salute__dem_suave, p_salute__fasc_suave],
        [p_salute__dem_awk, p_salute__fasc_awk],
    ])[demeanor, ideology]
    
    ### P(Gesture | Ideology, Demeanor)
    return jnp.array([1 - p_salute, p_salute])[gesture]

@memo
def speech_simulation[
    _g: Gesture, 
    _i: Ideology, 
    _d: Demeanor
](prior_fasc=0.5):
    observer: knows(_d, _i, _g)
    observer: thinks[
        broker: given(d in Demeanor, wpp=1),
        broker: chooses(i in Ideology, wpp=(
            prior_fasc if i == {Ideology.FASCISM} 
            else 1 - prior_fasc
        )),
        broker: chooses(g in Gesture, wpp=gesture_pmf(g, i, d)),
    ]

    return observer[ 
        Pr[
            broker.g == _g
            and broker.i == _i
            and broker.d == _d
        ] 
    ]



@memo
def speech_observation[
    _g: Gesture, 
    _i: Ideology, 
    _d: Demeanor
](prior_fasc=0.5):
    observer: knows(_d, _i, _g)
    observer: thinks[
        broker : given(d in Demeanor, wpp=1),
        broker : chooses(i in Ideology, wpp=(
            prior_fasc if i == {Ideology.FASCISM} 
            else 1 - prior_fasc
        )),
        broker : chooses(g in Gesture, wpp=gesture_pmf(g, i, d)),
    ]

    ### observe gesture ###
    observer: observes[broker.g] is _g

    return observer[
        Pr[
            ### replace _g with {Gesture.SALUTE} to ignore
            ### probs for when gesture something other than salute
            broker.g == _g
            and broker.i == _i
            and broker.d == _d
        ]
    ]


###
# How do these two observers' priors affect their belief updates 
# when they observe the gesture?
###

### observer 1
print(
    f"Observer 1 who, before observing the speech, "
    "had a slight positive belief about whether the broker "
    "is pro-fascism or pro-democracy"
)
res1 = speech_observation(
    prior_fasc=0.6, 
    print_table=True, return_aux=True, return_xarray=True)

print("\n\n")

### observer 2
print(
    f"Observer 2 who, before observing the speech, "
    "thought that the broker being pro-fascism was unlikely"
)
res2 = speech_observation(
    prior_fasc=0.01, 
    print_table=True, return_aux=True, return_xarray=True)



Observer 1 who, before observing the speech, had a slight positive belief about whether the broker is pro-fascism or pro-democracy
+---------------+--------------+--------------+---------------------------------+
| _g: Gesture   | _i: Ideology | _d: Demeanor | speech_observation[_g, _i, _d]  |
+---------------+--------------+--------------+---------------------------------+
| SOMETHINGELSE | DEMOCRACY    | SUAVE        | 0.5444444417953491              |
| SOMETHINGELSE | DEMOCRACY    | AWKWARD      | 0.3888888657093048              |
| SOMETHINGELSE | FASCISM      | SUAVE        | 0.02499997802078724             |
| SOMETHINGELSE | FASCISM      | AWKWARD      | 0.041666675359010696            |
| SALUTE        | DEMOCRACY    | SUAVE        | 0.006250000558793545            |
| SALUTE        | DEMOCRACY    | AWKWARD      | 0.0937500074505806              |
| SALUTE        | FASCISM      | SUAVE        | 0.45468753576278687             |
| SALUTE        | FASCISM      | AWKWARD      | 0

4. Extend the model in some fashion. You could add more causes, or more types of observations. You could model how inference is affected by observing one gesture (which could be more easily explained away as noisy movement production) versus multiple similar gestures (which could imply a deliberate signal). You could convert a binary variable into a discretized linear variable (e.g. turn Gesture into a perceptual similarity metric that express how confusable a gesture is with a fascist salute). Maybe the observers didn’t see the video but rather heard about it from someone they have differing degrees of trust in (see Jaynes, 2003, and also the exercise below) – perhaps the observers think the person tends to be hyperbolic or understated, or maybe the person is in their ingroup or outgroup.



5. Jaynes (2003, Chapter 5, Section 3) describes how the same data can cause observers’ opinions to diverge. Extend the political power broker model above so that the observers update their beliefs in opposite directions given the same data. Describe why these changes lead to belief polarization.

    For an empirical study that applies these ideas to actual behavior, see Botvinik-Nezer et al. (2023).

Here we extend the model by adding the variable 

import jax
import jax.numpy as jnp
from memo import memo
from enum import IntEnum
from xarray import DataArray

class Gesture(IntEnum):
    SOMETHINGELSE = 0
    SALUTE = 1

class Ideology(IntEnum):
    DEMOCRACY = 0
    FASCISM = 1

class Demeanor(IntEnum):
    SUAVE = 0
    AWKWARD = 1

class Information(IntEnum):
    OBSERVED = 0 # has no additonal effect (same as our default model) 
    HEARD_d = 1 ## hearing the interpretation from a democrat -> positive influence in the belief that the broker is pro-fascism, negative influence in the belief that the broker is pro-democracy
    HEARD_c = 2 ## hearing the interpretation from a conservative -> positive influence in the belief that the broker is pro-demo, negative influence in the belief that the broker is pro-fascism

@jax.jit
def gesture_pmf(gesture, ideology, demeanor):
    ### P(Gesture=SALUTE | Ideology=DEMOCRACY, Demeanor=SUAVE, Information=OBSERVED)
    p_salute__dem_suave_heardd = 0.001
    ### P(Gesture=SALUTE | Ideology=DEMOCRACY, Demeanor=AWKWARD, Information=OBSERVED)
    p_salute__dem_awk_heardd = 0.2
    ### P(Gesture=SALUTE | Ideology=FASCISM, Demeanor=SUAVE, Information=OBSERVED)
    p_salute__fasc_suave_heardd = 0.9
    ### P(Gesture=SALUTE | Ideology=FASCISM, Demeanor=AWKWARD, Information=OBSERVED)
    p_salute__fasc_awk_heardd = 0.92
    
    
    ### P(Gesture=SALUTE | Ideology=DEMOCRACY, Demeanor=SUAVE, Information=HEARD_d)
    p_salute__dem_suave = 0.001
    ### P(Gesture=SALUTE | Ideology=DEMOCRACY, Demeanor=AWKWARD, Information=HEARD_d)
    p_salute__dem_awk = 0.3
    ### P(Gesture=SALUTE | Ideology=FASCISM, Demeanor=SUAVE, Information=HEARD_d)
    p_salute__fasc_suave = 0.8
    ### P(Gesture=SALUTE | Ideology=FASCISM, Demeanor=AWKWARD, Information=HEARD_d)
    p_salute__fasc_awk = 0.92 

    
     
    ### P(Gesture=SALUTE | Ideology=DEMOCRACY, Demeanor=SUAVE, Information=HEARD_c)
    p_salute__dem_suave = 0.001
    ### P(Gesture=SALUTE | Ideology=DEMOCRACY, Demeanor=AWKWARD, Information=HEARD_c)
    p_salute__dem_awk = 0.3
    ### P(Gesture=SALUTE | Ideology=FASCISM, Demeanor=SUAVE, Information=HEARD_c)
    p_salute__fasc_suave = 0.8
    ### P(Gesture=SALUTE | Ideology=FASCISM, Demeanor=AWKWARD, Information=HEARD_c)
    p_salute__fasc_awk = 0.92 
    

    ### P(Gesture=SALUTE | Ideology, Demeanor)
    p_salute = jnp.array([
        [p_salute__dem_suave, p_salute__fasc_suave],
        [p_salute__dem_awk, p_salute__fasc_awk],
    ])[demeanor, ideology]
    
    ### P(Gesture | Ideology, Demeanor)
    return jnp.array([1 - p_salute, p_salute])[gesture]

@memo
def speech_simulation[_g: Gesture, _i: Ideology, _d: Demeanor, _inf: Information](prior_fasc=0.5):
    observer: knows(_d, _i, _g, _inf)
    observer: thinks[
        broker: given(d in Demeanor, wpp=1),
        broker: chooses(i in Ideology, wpp=prior_fasc if i == {Ideology.FASCISM} else 1 - prior_fasc),
        broker: chooses(g in Gesture, wpp=gesture_pmf(g, i, d)),
        broker: chooses(information in Information, wpp=1), 
    ]

    return observer[ 
        Pr[
            broker.g == _g
            and broker.i == _i
            and broker.d == _d
            and broker.inf = _inf
        ] 
    ]

### observer 1
print(f"Observer 1, with a uniform prior belief about whether the broker is pro-fascism or pro-democracy")
res1joint = speech_simulation(prior_fasc=0.5, print_table=True, return_aux=True, return_xarray=True)
print("\n\n")

### observer 2
print(f"Observer 2, who thinks that the broker being pro-fascism is unlikely")
res2joint = speech_simulation(prior_fasc=0.01, print_table=True, return_aux=True, return_xarray=True)